In [2]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas
import xarray
import cftime
import tensorflow as tf

/Users/paulag/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
datapath = 'nino34_monthly.nc'
nino34 = xarray.open_dataset(datapath, decode_times = False)
print(nino34)
nino34 = np.array(nino34['nino34'].values)

<xarray.Dataset>
Dimensions:         (bounds: 2, month: 12, time: 7800)
Coordinates:
  * time            (time) float64 15.5 45.0 74.5 ... 2.372e+05 2.372e+05
Dimensions without coordinates: bounds, month
Data variables:
    nino34          (time) float64 ...
    time_bnds       (time, bounds) float64 ...
    areacello       float32 ...
    days_per_month  (month) int32 ...


In [4]:
def ONI(nino34, m = 3):
    oni = np.array(nino34)
    length = nino34.shape[0]
    for i in range(length):
        oni[i] = np.mean(nino34[max(0, (i - m + 1)) : min((i + 1), length)])
    return oni

In [5]:
oni = ONI(nino34)

In [6]:
def climatology(nino34):
    clm = np.zeros(12)
    length = nino34.shape[0]
    for month in range(12):
        section = [12 * i + month for i in range(length // 12)]
        clm[month] = np.mean(nino34[section])
    return clm

In [7]:
clm = climatology(nino34)

In [8]:
def SST_anomaly(nino34, clm):
    anm = np.array(nino34)
    length = nino34.shape[0]
    for i in range(length):
        anm[i] = nino34[i] - clm[i % 12]
    return anm

In [9]:
anm = SST_anomaly(nino34, clm)
oanm = ONI(anm)

# Data Preparation

In [10]:
T = 6                       # prediction timeline
H = 48                      # history used for prediction
include_month = 1           # 1 if we use the month as a feature, 0 otherwise
n_classes = 3               # number of classes (El Nino, El Nina, No Event)
threshold = 0.5         
signal = np.array(nino34)   # data used for training/testing
length = signal.shape[0]    # number of data points
size = length - H - T       # effective dataset size

In [11]:
# create the 'history matrix'
data = np.ndarray((size, H + include_month))
for i in range(size):
    if(include_month == False):
        data[i] = signal[i:(i + H)]
    else:
        data[i] = np.append(signal[i:(i + H)], (i + H + T) % 12)

# label El Nino as 2, El Nina as 0 and no event as 1
labels = np.ndarray((size))
for i in range(length - H - T):
    if(oanm[i + H + T] >= threshold):
        labels[i] = 2
    elif(oanm[i + H + T] <= -threshold):
        labels[i] = 0
    else:
        labels[i] = 1

In [12]:
np.random.seed(0)

split = size // 10      
shuffle = np.random.permutation(size)
train_ind = np.array(shuffle[0: 8 * split])
val_ind = np.array(shuffle[(8 * split + 1): 9 * split])
test_ind = np.array(shuffle[(9 * split + 1): size])

train = np.array(data[train_ind])
train_labels = np.array(labels[train_ind])

val = np.array(data[val_ind])
val_labels = np.array(labels[val_ind])

test = np.array(data[test_ind])
test_labels = np.array(labels[test_ind])

# Normalization and Label One-Hot Encoding

We normalize each feature individually because of the difference in scale between month number and temperature values.

In [26]:
mean = np.mean(train, axis = 0)
std = np.std(train, axis = 0)

train_n = np.divide(
    train - np.outer(np.ones(train.shape[0]), mean),
    np.outer(np.ones(train.shape[0]), std))

val_n = np.divide(
    val - np.outer(np.ones(val.shape[0]), mean),
    np.outer(np.ones(val.shape[0]), std))

test_n = np.divide(
    test - np.outer(np.ones(test.shape[0]), mean),
    np.outer(np.ones(test.shape[0]), std))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    encoded_train_labels = tf.one_hot(train_labels, depth = n_classes).eval()
    encoded_val_labels = tf.one_hot(val_labels, depth = n_classes).eval()
    encoded_test_labels = tf.one_hot(test_labels, depth = n_classes).eval()

# Training a Network using the month feature

In [32]:
epochs = 40
dropout_rate = 0.3
N = 4 * H

In [29]:
model = tf.keras.models.Sequential([
              tf.keras.layers.Dense(N, activation=tf.nn.relu),
              tf.keras.layers.Dropout(dropout_rate),
              tf.keras.layers.Dense(N, activation=tf.nn.relu),
              tf.keras.layers.Dropout(dropout_rate),
              tf.keras.layers.Dense(N, activation=tf.nn.relu),
              tf.keras.layers.Dropout(dropout_rate),
              tf.keras.layers.Dense(n_classes, activation=tf.nn.softmax)
            ])

model.reset_states()

model.compile(optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

model.fit(train_n, encoded_train_labels, epochs = epochs)
model.evaluate(val_n, encoded_val_labels)

Epoch 1/40
6192/6192 [==============================] - 2s 261us/step - loss: 0.9483 - acc: 0.5404
Epoch 2/40
6192/6192 [==============================] - 1s 161us/step - loss: 0.8703 - acc: 0.5887
Epoch 3/40
6192/6192 [==============================] - 1s 167us/step - loss: 0.8327 - acc: 0.6064
Epoch 4/40
6192/6192 [==============================] - 1s 162us/step - loss: 0.8137 - acc: 0.6184
Epoch 5/40
6192/6192 [==============================] - 1s 144us/step - loss: 0.7926 - acc: 0.6281
Epoch 6/40
6192/6192 [==============================] - 1s 132us/step - loss: 0.7683 - acc: 0.6442
Epoch 7/40
6192/6192 [==============================] - 1s 154us/step - loss: 0.7409 - acc: 0.6652
Epoch 8/40
6192/6192 [==============================] - 1s 143us/step - loss: 0.7098 - acc: 0.6757
Epoch 9/40
6192/6192 [==============================] - 1s 143us/step - loss: 0.6857 - acc: 0.6917
Epoch 10/40
6192/6192 [==============================] - 1s 134us/step - loss: 0.6540 - acc: 0.7051
Epoch 11/

[1.553181263638653, 0.6856403622443741]

The accuracy above suggests there are no issues with our method of normalization. We will now use this augmented data with the architecture chosen in Part III.

In [ ]:
epochs = 50
dropout_rate = 0.5
(N1, N2, N3) = (4 * H, 8 * H, 8 * H)

In [ ]:
model = tf.keras.models.Sequential([
              tf.keras.layers.Dense(N1, activation=tf.nn.relu),
              tf.keras.layers.Dropout(dropout_rate),
              tf.keras.layers.Dense(N2, activation=tf.nn.relu),
              tf.keras.layers.Dropout(dropout_rate),
              tf.keras.layers.Dense(N3, activation=tf.nn.relu),
              tf.keras.layers.Dropout(dropout_rate),
              tf.keras.layers.Dense(n_classes, activation=tf.nn.softmax)
            ])

model.reset_states()

model.compile(optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

model.fit(train_n, encoded_train_labels, epochs = epochs)
model.evaluate(val_n, encoded_val_labels)